In [ ]:
# Wilmar Perez

# Import the Python AWS SDK
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/core/boto3.html

import boto3
from boto3.dynamodb.conditions import Key
from pprint import pprint
from decimal import Decimal
import json

In [ ]:
# Create Table
def create_movie_table(table_name='Movies'):

    table = dynamodb_resource.create_table(
        TableName=table_name,
        KeySchema=[
            {
                'AttributeName': 'year',
                'KeyType': 'HASH'  # Partition key
            },
            {
                'AttributeName': 'title',
                'KeyType': 'RANGE'  # Sort key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'year',
                'AttributeType': 'N'
            },
            {
                'AttributeName': 'title',
                'AttributeType': 'S'
            },

        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    )
    return table

In [ ]:
# Add data (one movie)
def put_movie(title, year, plot, rating, table_name='Movies'):

    table = dynamodb_resource.Table(table_name)
    response = table.put_item(
       Item={
            'year': year,
            'title': title,
            'info': {
                'plot': plot,
                'rating': rating
            }
        }
    )
    return response

In [ ]:
# Add data (in bulk)
def load_movies(movies, table_name='Movies'):

    table = dynamodb_resource.Table(table_name)
    for movie in movies:
        year = int(movie['year'])
        title = movie['title']
        print("Adding movie:", year, title)
        table.put_item(Item=movie)

In [ ]:
def query_movies_by_year(year, table_name='Movies'):

    table = dynamodb_resource.Table(table_name)
    response = table.query(
        KeyConditionExpression=Key('year').eq(year)
    )
    return response['Items']

In [ ]:
## config

aws_profile = 'LOCAL_PROFILE'
aws_service = 'dynamodb'

# Note that the db is equivalent to the table
my_table = 'DB_NAME'

# JSON with movies
json_file_with_movies_data = 'moviedata.json'

In [ ]:
# We need a new session in order to use a profile different to the Default one
session = boto3.Session(profile_name = aws_profile)

In [ ]:
# Create resource and client
dynamodb_resource = session.resource(aws_service)
dynamodb_client = session.client(aws_service)

In [ ]:
# Let's review what tables (databases) we have
for table in dynamodb_resource.tables.all():
    print(table.name)

In [ ]:
# Let's create our database
movie_table = create_movie_table(table_name = my_table)

In [ ]:
# Let's check the status of the database
dynamodb_client.describe_table(TableName = my_table)

In [ ]:
# Let's review what tables (databases) we have again!
for table in dynamodb_resource.tables.all():
    print(table.name)

In [ ]:
# Let's add some data
movie_resp = put_movie("The Big New Most Boring Movie", 2022,
                           "Nothing happens at all.", 0,
                           table_name = my_table)

In [ ]:
# Did it work?
pprint(movie_resp, sort_dicts=False)

In [ ]:
# Let's add some data again but this time around a lot of it
with open("moviedata.json") as json_file:
    movie_list = json.load(json_file, parse_float=Decimal)
    
load_movies(movie_list, table_name = my_table)

In [ ]:
# Let's query the DB

query_year = 2010
print("Movies from {}".format(query_year))
movies = query_movies_by_year(query_year, table_name = my_table)

for movie in movies:
    print(movie['year'], ":", movie['title'])
    # print(movie)


In [ ]:
# Now, let's be evil and delete that table!
movie_table.delete()